In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
df = pd.read_csv('/content/olid-training-v1.0.tsv', sep='\t')

# Remove unnecessary columns
df = df.drop(['id', 'subtask_a', 'subtask_b'], axis=1)

# Drop missing values
df = df.dropna()

# Encode the labels
labels = {'OTH': 0, 'GRP': 1, 'IND': 2}
df['subtask_c'] = df['subtask_c'].apply(lambda x: labels[x])

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['tweet'], df['subtask_c'], test_size=0.2, random_state=42)

In [ ]:
# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize the training and validation sets
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.values
))

In [ ]:
# Batch the datasets
train_dataset = train_dataset.batch(16)
val_dataset = val_dataset.batch(16)

In [ ]:
# Define the base model
base_model = TFAutoModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Define the meta-learner model
meta_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
# Define the training loop
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
        meta_output = meta_model(base_output)
        loss = loss_fn(labels, meta_output)
    gradients = tape.gradient(loss, meta_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, meta_model.trainable_weights))
    return loss

In [ ]:
# Train the model
for epoch in range(20):
    for inputs, labels in train_dataset:
        loss = train_step(inputs, labels)
    print('Epoch', epoch+1, '- Loss:', loss.numpy())

Epoch 1 - Loss: 0.8617881
Epoch 2 - Loss: 0.83502215
Epoch 3 - Loss: 0.8114011
Epoch 4 - Loss: 0.8037483
Epoch 5 - Loss: 0.7768869
Epoch 6 - Loss: 0.7550928
Epoch 7 - Loss: 0.7185175
Epoch 8 - Loss: 0.68342876
Epoch 9 - Loss: 0.64673436
Epoch 10 - Loss: 0.6077912
Epoch 11 - Loss: 0.56479436
Epoch 12 - Loss: 0.51561195
Epoch 13 - Loss: 0.46192613
Epoch 14 - Loss: 0.41965663
Epoch 15 - Loss: 0.3611779
Epoch 16 - Loss: 0.31358314
Epoch 17 - Loss: 0.2764782
Epoch 18 - Loss: 0.23456965
Epoch 19 - Loss: 0.20718217
Epoch 20 - Loss: 0.16817378


In [ ]:
# Evaluate the model on the validation set
true_labels = []
predictions = []
for inputs, labels in val_dataset:
    base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
    meta_output = meta_model(base_output).numpy()
    predicted_labels = np.argmax(meta_output, axis=1)
    true_labels.extend(labels.numpy())
    predictions.extend(predicted_labels)

from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.21      0.04      0.07        73
           1       0.54      0.56      0.55       202
           2       0.78      0.86      0.82       501

    accuracy                           0.70       776
   macro avg       0.51      0.49      0.48       776
weighted avg       0.67      0.70      0.68       776

